### Flip and blur the test set images


In [1]:
import numpy as np
import keras
from keras.datasets import mnist 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [2]:
def resizing(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            train_data.append(resized_img)
     return train_data

def flip_ver(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            resized_img = cv2.flip(resized_img, 0)
            train_data.append(resized_img)
     return train_data

def flip_hor(mnist):
     train_data = []
     for img in mnist:
            resized_img = cv2.resize(img, (32, 32))
            resized_img = cv2.flip(resized_img, 1)
            train_data.append(resized_img)
     return train_data
newX_train = np.array(resizing(X_train))
verX_test = np.array(flip_ver(X_test))
horX_test = np.array(flip_hor(X_test))

In [3]:
from keras import backend
if keras.backend.image_data_format() == 'channels_first':
  newX_train = newX_train.reshape(len(newX_train), 1, 32, 32)
  verX_test = verX_test.reshape(len(verX_test), 1, 32, 32)
  horX_test = horX_test.reshape(len(horX_test), 1, 32, 32)
  input_shape = (1, 32, 32)
else:
  newX_train = newX_train.reshape(len(newX_train), 32, 32, 1)
  verX_test = verX_test.reshape(len(verX_test), 32, 32, 1)
  horX_test = horX_test.reshape(len(horX_test), 32, 32, 1)
  input_shape = (32, 32, 1)

In [4]:
from keras.utils import np_utils
def nor(dataset):
  dataset = dataset.astype('float32')
  dataset /= 255
  return dataset
newX_train = nor(newX_train)
verX_test = nor(verX_test)
horX_test = nor(horX_test)
Y_train = keras.utils.np_utils.to_categorical(Y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(Y_test, 10)

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = (2, 2)))
model.add(Flatten()) 
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 256)         295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 256)         5

In [6]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
history = model.fit(newX_train[:600], Y_train[:600], batch_size=32, validation_data=(verX_test[:100],Y_test[:100]), epochs=10, verbose=1)
model.evaluate(verX_test[:100], Y_test[:100], batch_size=32)[1]

Epoch 1/10
19/19 [==============================] - 39s 2s/step - loss: 0.0120 - accuracy: 0.9933 - val_loss: 5.7052 - val_accuracy: 0.3100
Epoch 2/10
19/19 [==============================] - 39s 2s/step - loss: 0.0279 - accuracy: 0.9950 - val_loss: 5.2995 - val_accuracy: 0.3100
Epoch 3/10
19/19 [==============================] - 38s 2s/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 5.1498 - val_accuracy: 0.3200
Epoch 4/10
19/19 [==============================] - 39s 2s/step - loss: 0.0053 - accuracy: 0.9983 - val_loss: 5.0132 - val_accuracy: 0.3300
Epoch 5/10
19/19 [==============================] - 39s 2s/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 5.2880 - val_accuracy: 0.3200
Epoch 6/10
19/19 [==============================] - 38s 2s/step - loss: 5.2118e-04 - accuracy: 1.0000 - val_loss: 5.5194 - val_accuracy: 0.3200
Epoch 7/10
19/19 [==============================] - 39s 2s/step - loss: 3.0471e-04 - accuracy: 1.0000 - val_loss: 5.6687 - val_accuracy: 0.3200
Epoch 8/10
1

0.3199999928474426

##### The vertical flip test accuracy is 0.32, which is greatly reduced compared to previous one.


In [9]:
history = model.fit(newX_train[:600], Y_train[:600], batch_size=32, validation_data=(horX_test[:100], Y_test[:100]), epochs=10, verbose=1)
model.evaluate(horX_test[:100], Y_test[:100], batch_size=32)[1]

Epoch 1/10
19/19 [==============================] - 40s 2s/step - loss: 0.0336 - accuracy: 0.9900 - val_loss: 5.0640 - val_accuracy: 0.3000
Epoch 2/10
19/19 [==============================] - 40s 2s/step - loss: 0.0302 - accuracy: 0.9883 - val_loss: 5.3502 - val_accuracy: 0.3000
Epoch 3/10
19/19 [==============================] - 40s 2s/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 5.0121 - val_accuracy: 0.3900
Epoch 4/10
19/19 [==============================] - 39s 2s/step - loss: 9.9937e-04 - accuracy: 1.0000 - val_loss: 5.5378 - val_accuracy: 0.3700
Epoch 5/10
19/19 [==============================] - 42s 2s/step - loss: 5.6863e-04 - accuracy: 1.0000 - val_loss: 5.7347 - val_accuracy: 0.3700
Epoch 6/10
19/19 [==============================] - 39s 2s/step - loss: 4.3156e-04 - accuracy: 1.0000 - val_loss: 5.8872 - val_accuracy: 0.3700
Epoch 7/10
19/19 [==============================] - 38s 2s/step - loss: 3.5822e-04 - accuracy: 1.0000 - val_loss: 6.0057 - val_accuracy: 0.3800
Epoc

0.3799999952316284

##### The vertical flip test accuracy is 0.38, which is greatly reduced compared to previous one.